# YOLOv3 Crater Detection Algorithm Workflow
---
### Giles Matthews

## Setup

In [ ]:
print("---  Cloning repository  ---\n")
! git clone https://github.com/acse-gcm20/YOLOv3-CDA
print("\n---  Installing dependencies  ---\n")
! pip install -r /content/YOLOv3-CDA/requirements.txt
print("\nNow restart the Runtime before proceeding")

In [ ]:
%cd /content/YOLOv3-CDA

import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from sklearn.metrics import auc

from pytorchyolo.train import run
from pytorchyolo.test import evaluate_model_file
from pytorchyolo.detect import detect_directory

/content/YOLOv3-CDA


In [ ]:
# Download weights pretrained on COCO
! wget -c "https://pjreddie.com/media/files/yolov3.weights" --header "Referer: pjreddie.com"

--2021-07-12 18:10:22--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  23.3MB/s    in 11s     

2021-07-12 18:10:34 (21.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



## Data Organisation

In [ ]:
# Organise THEMIS data intro training, validation and test sets
imageFolder = "/content/YOLOv3-CDA/data/images/"
labelFolder = "/content/YOLOv3-CDA/data/labels/"

pathsFile = "/content/YOLOv3-CDA/data/all.txt"
trainFile = "/content/YOLOv3-CDA/data/train.txt"
valFile = "/content/YOLOv3-CDA/data/valid.txt"
testFile = "/content/YOLOv3-CDA/data/test.txt"

# Walk through image folder and write paths to all.txt, also count number of files
with open(pathsFile, "w") as all:
    n = 0
    for root, dirs, files in os.walk(imageFolder):
        for file in files:
            all.write(imageFolder+file+"\n")
            n += 1

# 90% of data into training set
trainSize = int(n * 0.9)
# 10% of training set into validation set
valSize = int(trainSize * 0.1)
trainSize -= valSize
# 10% of data into test set
testSize = n - (trainSize + valSize)

# Ensure dataset sizes are correct
try:
    assert n == trainSize + valSize + testSize
    print("Total:", n, "\nTraining Size:", trainSize, "\nValidation Size:", valSize, "\nTest Size:", testSize)
except:
    print("Invalid Split")

# Get random indices lists according to dataset sizes
train_idx, val_idx, test_idx = torch.utils.data.random_split(range(n), [trainSize, valSize, testSize], generator=torch.Generator().manual_seed(42))

Total: 3556 
Training Size: 2880 
Validation Size: 320 
Test Size: 356


In [ ]:
# Utility function to write lists to a text file
def list_to_file(list_array, filename):
    dims = len(np.shape(list_array))
    
    if dims == 1:
        rows = len(list_array)
        cols = 1
    else:
        rows = len(list_array[0])
        cols = len(list_array)

    with open(filename, 'w') as f:
        for row in range(rows):
            vals = [list_array[col][row] for col in range(cols)]
            f.write(("{} "*cols+'\n').format(*vals))

In [ ]:
# Organise image path files

# Get list of all image paths
with open(pathsFile, "r") as paths:
    pathsList = paths.readlines()

# Store train/val/test paths as lists
trainList = [pathsList[i].rstrip('\n') for i in train_idx]
valList = [pathsList[i].rstrip('\n') for i in val_idx]
testList = [pathsList[i].rstrip('\n') for i in test_idx]

# Write path lists to files
list_to_file([trainList], trainFile)
list_to_file([valList], valFile)
list_to_file([testList], testFile)

print("Training paths:\t\t", trainFile, "\nValidation paths:\t", valFile, "\nTest paths:\t\t", testFile)

Training paths:		 /content/YOLOv3-CDA/data/train.txt 
Validation paths:	 /content/YOLOv3-CDA/data/valid.txt 
Test paths:		 /content/YOLOv3-CDA/data/test.txt


In [1]:
# Pull any edits
! git pull https://github.com/acse-gcm20/YOLOv3-CDA
print("\nNow restart the Runtime and re-import modules before proceeding")

fatal: not a git repository (or any of the parent directories): .git

Now restart the Runtime before proceeding


## Training

In [ ]:
# Run training sequence
run(epochs=50, pretrained_weights='yolov3.weights')

Training

Epochs: 50, Seed: 42


Training Epoch 0:   0%|          | 0/180 [00:00<?, ?it/s]


Epoch: 0
---- Training Model ----


Training Epoch 0: 100%|██████████| 180/180 [02:09<00:00,  1.39it/s]


---- Validating Model ----


...


---Evaluation---
Training Loss 0.6425876617431641
Validation loss: 0.7725457549095154

Epoch: 49
---- Training Model ----


Training Epoch 49: 100%|██████████| 180/180 [02:07<00:00,  1.41it/s]


---- Validating Model ----



Computing AP: 100%|██████████| 1/1 [00:00<00:00, 272.45it/s]


---- mAP 0.74511 ----
Precision: 0.7379095163806553, Recall: 0.8127147766323024
---- Saving checkpoint to: 'checkpoints/yolov3_ckpt_49.pth' ----

---Evaluation---
Training Loss 0.7512170672416687
Validation loss: 0.6778684258460999


In [ ]:
# Download file
files.download('checkpoints/yolov3_ckpt_49.pth')

In [ ]:
# Upload file
files.upload()

## Analysis and Plotting

In [ ]:
# Plot statistics

def plot_stats(stats_file):
    training_losses = []
    validation_losses = []
    precision_vals = []
    recall_vals = []

    with open(stats_file) as f:
        for line in f:
            tl, vl, precision, recall = line.rstrip("\n").split()
            training_losses.append(float(tl))
            validation_losses.append(float(vl))
            precision_vals.append(float(precision))
            recall_vals.append(float(recall))

    epochs = len(training_losses)

    fig, axs = plt.subplots(2, 2, figsize=(16, 16))

    axs[0,0].plot(range(1, epochs+1), training_losses, label="Training Loss")
    axs[0,0].plot(range(1, epochs+1), validation_losses, label="Validation Loss")
    axs[0,0].set_xlim(1, epochs)
    axs[0,0].set_xlabel("Epoch")
    axs[0,0].set_ylabel("Loss")
    axs[0,0].legend()
    axs[0,0].grid(True)

    axs[0,1].scatter(recall_vals, precision_vals)
    axs[0,1].set_xlim(-0.1, 1.1)
    axs[0,1].set_ylim(-0.1, 1.1)
    axs[0,1].set_xlabel("Recall")
    axs[0,1].set_ylabel("Precision")
    axs[0,1].grid(True)

    axs[1,0].scatter(range(1, epochs+1), recall_vals)
    axs[1,0].set_xlim(1, epochs)
    axs[1,0].set_ylim(-0.1, 1.1)
    axs[1,0].set_xlabel("Epoch")
    axs[1,0].set_ylabel("Recall")
    axs[1,0].grid(True)

    axs[1,1].scatter(range(1, epochs+1), precision_vals)
    axs[1,1].set_xlim(1, epochs)
    axs[1,1].set_ylim(-0.1, 1.1)
    axs[1,1].set_xlabel("Epoch")
    axs[1,1].set_ylabel("Precision")
    axs[1,1].grid(True)

In [ ]:
plot_stats('stats.txt')

In [ ]:
class PRcurve:
    def __init__(self, weights_path, start=0.05, stop=0.5, n=10):
        self.confs = np.linspace(start, stop, num=n)
        self.precisions = []
        self.recalls = []

        for i, conf in enumerate(self.confs):
            print("\n%i/%i: Confidence Threshold = %.3f" % (i+1, n, conf))
            metrics = evaluate_model_file('yolov3.cfg', weights_path, 'data/valid.txt', ['crater'], batch_size=32, n_cpu=2, conf_thres=conf, verbose=False)
            self.precisions.append(float(metrics[0]))
            self.recalls.append(float(metrics[1]))
            print("Precision = %.3f, Recall = %.3f" % (metrics[0], metrics[1]))

        list_to_file([self.precisions, self.recalls], "PRstats.txt")
        self.auc_score = auc(self.recalls, self.precisions)

    def plot(self):
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))

        ax.plot(self.recalls, self.precisions)
        ax.set_title("Precision-Recall Curve, AUC = %.2f" % self.auc_score)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_xlabel("Recall")
        ax.set_ylabel("Precision")
        ax.grid(True)

In [ ]:
# 50 epochs not pre-trained
set1 = PRcurve('/content/drive/MyDrive/CDA/ckpt_50.pth')
set1.plot()

In [ ]:
# 50 epochs pre-trained
set2 = PRcurve('/content/drive/MyDrive/CDA/ckpt_50_pt.pth')
set2.plot()

## Detections

In [ ]:
# Perform detections on a directory

#directory = '/content/YOLOv3-CDA/data/samples/'    # Sample Directory
directory = '/content/YOLOv3-CDA/data/HiRISE/'      # Tiles from HiRISE

with open("/content/YOLOv3-CDA/data/sample_paths.txt", "w") as samples:
    n = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            samples.write(directory+file+"\n")
            n += 1

detect_directory('yolov3.cfg', '/content/drive/MyDrive/CDA/ckpt_50_pt.pth', directory, ['crater'], 'output', batch_size=1, n_cpu=2, conf_thres=0.1)